In [ ]:
%pip install google-generativeai
%pip install python-dotenv

In [ ]:
import os
import json
import google.generativeai as genai
from dotenv import load_dotenv

# === CONFIGURACIÓN ===
CARPETA_ENTRADA = "textos"
CARPETA_SALIDA = "resultados"
ARCHIVO_LOG = "log_errores.txt"

# Inicializar modelo
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

# Or use `os.getenv('GEMINI_API_KEY')` to fetch an environment variable.
#GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")
#GEMINI_API_KEY = 'un-api-valido'
genai.configure(api_key=GEMINI_API_KEY)


#model = genai.GenerativeModel("gemini-1.5-flash")
#model = genai.GenerativeModel("gemini-2.0-flash-lite")
#model = genai.GenerativeModel("gemini-2.0-flash")
#model = genai.GenerativeModel("gemini-2.0-flash-exp")

# Inicializar modelo con parámetros ajustados para mejorar precisión y robustez
model = genai.GenerativeModel(
    model_name="gemini-2.0-flash-lite",
    generation_config={
        "temperature": 0.8,
        "top_p": 0.8,
        "top_k": 40,
        "max_output_tokens": 2048,
        "candidate_count": 1
    }
)

In [73]:
# Asegurar que exista la carpeta de salida
os.makedirs(CARPETA_SALIDA, exist_ok=True)

# Inicializar archivo de log
with open(ARCHIVO_LOG, "w", encoding="utf-8") as log:
    log.write("=== LOG DE ERRORES ===\n")

# Archivos de entrada
archivos_txt = [f for f in os.listdir(CARPETA_ENTRADA) if f.endswith(".txt")]

# Token acumulado
total_tokens_global = 0

In [74]:
# Procesamiento
for archivo in archivos_txt:
    ruta_entrada = os.path.join(CARPETA_ENTRADA, archivo)
    ruta_salida = os.path.join(CARPETA_SALIDA, f"{os.path.splitext(archivo)[0]}.json")

    with open(ruta_entrada, "r", encoding="utf-8") as f:
        texto = f.read()

    prompt = f"""
    Clasificá el siguiente texto según su orientación política en Argentina como 'izquierda', 
    'derecha' o 'neutral' en ultima instancia, la prioridad es derecha o izquierda, pero si es muy incierto entonces se dice neutral. Para el campo 'texto_id' usá el nombre
    del archivo sin extensión. 
    Devolvé la respuesta en formato JSON con las siguientes claves:

    - \"texto_id\": nombre identificador (usá el nombre del archivo sin extensión)
    - \"clasificacion\": 'izquierda', 'derecha' o 'neutral'
    - \"confianza\": alta, media o baja
    - \"motivos\": lista de 3 a 5 razones
    - \"entidades_mencionadas\": lista con nombres propios políticos o de figuras públicas 
    mencionadas

    Texto:
    \"\"\"
    {texto}
    \"\"\"
    """

    try:
        response = model.generate_content(prompt)

        raw = response.text.strip()
        if raw.startswith("```json"):
            raw = raw.removeprefix("```json").removesuffix("```").strip()

        total_tokens = None
        if hasattr(response, "usage_metadata") and response.usage_metadata:
            total_tokens = response.usage_metadata.total_token_count
            total_tokens_global += total_tokens or 0

        resultado = json.loads(raw)

        if "texto_id" not in resultado:
            resultado["texto_id"] = os.path.splitext(archivo)[0]

        resultado["tokens_procesados"] = total_tokens

    except Exception as e:
        resultado = {
            "texto_id": os.path.splitext(archivo)[0],
            "error": str(e),
            "raw_response": getattr(response, "text", "Sin respuesta"),
            "tokens_procesados": None
        }
        with open(ARCHIVO_LOG, "a", encoding="utf-8") as log:
            log.write(f"❌ Error en {archivo}: {str(e)}\n")

    with open(ruta_salida, "w", encoding="utf-8") as f:
        json.dump(resultado, f, ensure_ascii=False, indent=2)

    print(f"✅ Procesado: {archivo} | Tokens: {resultado.get('tokens_procesados', 'N/A')}")

# Mostrar total de tokens usados
print(f"\n🧠 Total global de tokens procesados: {total_tokens_global}")


✅ Procesado: #25demayoenradio10 - banda militar en el centro cultural kirchner.txt | Tokens: 546
✅ Procesado: #25demayoenradio10 - jorge capitanich mano a mano con jorge rial.txt | Tokens: 2999
✅ Procesado: #25demayoenradio10 - los pampas en el centro cultural kirchner.txt | Tokens: 925
✅ Procesado: #25demayoenradio10 - martín bonavetti con gustavo sylvestre y jorge rial.txt | Tokens: 1098
✅ Procesado: #25demayoenradio10 - tristán bauer, ministro de cultura, mano a mano con jorge rial.txt | Tokens: 1403
✅ Procesado: #argenzuela -  audios exclusivos de la entrevista de jey mammon con rial.txt | Tokens: 3167
✅ Procesado: #argenzuela -  diego giuliano, ministro de transporte de la nación.txt | Tokens: 1378
✅ Procesado: #argenzuela -  federico fagioli, senador bonaerense, referente del patria grande.txt | Tokens: 2232
✅ Procesado: #argenzuela -  miguel angel pichetto sobre el asado en olivos y el festejo político de javier milei.txt | Tokens: 2372
✅ Procesado: #argenzuela -  raúl jalil le 